In [9]:
import pandas
from pytz import timezone
from datetime import datetime, timedelta
from ipywidgets import interact, Dropdown
from plotly.offline import download_plotlyjs,init_notebook_mode, iplot
from plotly.graph_objs import Candlestick

init_notebook_mode(connected=True)

symbols = ['ES','TU','UB']
periods = {
'10 minutes': lambda dts: dts.floor('10T'),
'5 minutes': lambda dts: dts.floor('5T'),
'1 minute': lambda dts: dts.floor('1T'),
'30 seconds': lambda dts: dts,
}
                                      
events = pandas.read_csv('events.csv',names=['dt','curr','desc','imp','a','f','p','key'])
events['dt'] = pandas.to_datetime(events['dt'],format="%Y-%m-%dT%H:%M:%S").dt.tz_localize('UTC').dt.tz_convert("US/Central").dt.tz_localize(None)
grouped = events.groupby('key')

In [12]:
def plot(data, event, groupfunc):
    data.index = groupfunc(data.index)
    ohlc = data.groupby(data.index).agg({'o':['first'],'h':['max'],'l':['min'],'c':['last']})
    candles = Candlestick(x=ohlc.index
                          ,open=ohlc['o']['first']
                          ,high=ohlc['h']['max']
                          ,low=ohlc['l']['min']
                          ,close=ohlc['c']['last']
                          ,whiskerwidth=1)
    layout = {
    'title':event.desc,
    'shapes': [
    {
    'type': 'line', 'x0': event[0], 'y0': 0, 'x1': event[0], 'y1': 1,
    'xref': 'x', 'yref': 'paper', 'line': {'color': 'rgb(0, 0, 255)'}
    }
    ]
    }

    return iplot({'data':[candles],'layout':layout})
    
df = None
sym = None
def load_data(symbol,dt,window=60): # window : one side window size in mins
    global df, sym
    if df is None or symbol != sym:
        sym = symbol
        df = pandas.read_hdf('ticks30s.h5',symbol.lower())
    before = (dt - timedelta(minutes=window)).floor('D')
    after = (dt + timedelta(minutes=window)).ceil('D')
    return df[(before <= df.index) & (df.index <= after)].copy()

def onSelectEvent(*args):
    group = grouped.get_group(evdrop.value).set_index('dt')
    dtstrs = group.index.strftime('%b %d %Y, %r')
    dtdrop.value = None
    dtdrop.options = reversed(list(zip(dtstrs, group.itertuples())))
    
def run(sb,pd,_,ev):
    if ev is not None:
        data = load_data(sb,ev[0])
        ret = plot(data,ev,pd)
        return ret

In [13]:
sbdrop = Dropdown(options=symbols,description="Security:",value='UB')
pddrop = Dropdown(options=periods,description="Period:")
evdrop = Dropdown(options=grouped.groups.keys(),description="Event:",value=None);
dtdrop = Dropdown(description="Date:")
evdrop.observe(onSelectEvent, 'value')

_ = interact(run, sb=sbdrop, pd=pddrop, _=evdrop, ev=dtdrop)

interactive(children=(Dropdown(description='Security:', index=2, options=('ES', 'TU', 'UB'), value='UB'), Drop…